In [1]:
import pandas as pd
import datetime
import re
from re import sub
import warnings

from sklearn import preprocessing
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression

import numpy as np

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

warnings.filterwarnings("ignore")

import seaborn as sns
# import matplotlib as plt
import matplotlib.pyplot as plt

from yellowbrick.features import rank2d
from yellowbrick.classifier import confusion_matrix
from yellowbrick.classifier import classification_report
from yellowbrick.regressor import prediction_error, ResidualsPlot

# Change pandas viewing options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielacollaguazo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Importing Reviews

In [2]:
# importing the reviews text with polarity
df_reviews_w_polarity = pd.read_csv('../variable_exploration/dc/output/reviews_with_sentiment_and_lang.csv')

## Selecting reviews only in English

In [3]:
# getting only reviews in English
df_rev_eng=df_reviews_w_polarity[df_reviews_w_polarity.review_lang=='en']

In [4]:
df_rev_eng.review_lang.value_counts(dropna=False)

en    1009583
Name: review_lang, dtype: int64

In [5]:
df_rev_eng.shape

(1009583, 8)

## Creating features target dataset

In [6]:
df_rev_eng = df_rev_eng[['listing_id', 'comments']]

In [9]:
# merging colums wit normalized ratings and the price column
df_temp = pd.merge(left=df_rev_eng, right=df_listings, on='listing_id')

# remove null values
df_temp = df_temp.dropna()

df_temp = df_temp.set_index('listing_id')

In [10]:
df_temp.head()

,comments,price
listing_id,,
2060,"very nice neighborhood,close enough to ""A"" tra...",100.0
2595,I've stayed with my friend at the Midtown Cast...,225.0
2595,"We've been staying here for about 9 nights, en...",225.0
2595,We had a wonderful stay at Jennifer's charming...,225.0
2595,Hi to everyone!\r\nWould say our greatest comp...,225.0


## Text pre-processing

### Cleaning

In [11]:
def text_cleaner(s):
#     print(s)
#     Replace special characters with ''
    stripped = re.sub('[^\w\s]','',s)
    stripped = re.sub('_','',stripped)
    
#     Change any whitespace to one space
    stripped = re.sub('\s',' ', stripped)
    
#     Remove numbers
    stripped = re.sub('[\d]','',stripped)
    stripped = stripped.lower()
#     To lowercase
    
#     Remove start and end white spaces
    stripped = stripped.strip()
    
    return stripped

In [12]:
df_temp['cleaned_comments'] = df_temp.apply(lambda row: text_cleaner(row['comments']), axis=1)